# Pivot and Pivot Table 

In [1]:
%autosave 0

Autosave disabled


In [2]:
# print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

# Pivot

Create the data

In [3]:
from collections import OrderedDict

table = OrderedDict((
    ("Product", ['Car', 'Car', 'Car', 'Motorcycle', 'Motorcycle', 'Motorcycle']),
    ('CustomerType',['Dealer', 'Member', 'Non-member', 'Dealer', 'Member', 'Non-member']),
    ('USD',  ['170', '180', '200', '75', '85', '100']),
    ('EU',   ['160', '170', '185', '70', '80', '90'])
))

Note: An **OrderedDict** is a Python dictionary subclass that remembers the order that keys were first inserted. 

In [4]:
table

OrderedDict([('Product',
              ['Car', 'Car', 'Car', 'Motorcycle', 'Motorcycle', 'Motorcycle']),
             ('CustomerType',
              ['Dealer',
               'Member',
               'Non-member',
               'Dealer',
               'Member',
               'Non-member']),
             ('USD', ['170', '180', '200', '75', '85', '100']),
             ('EU', ['160', '170', '185', '70', '80', '90'])])

In [5]:
type(table)

collections.OrderedDict

Need to convert the dictionary into a DataFrame first.

In [6]:
df = pd.DataFrame(table)

In [7]:
df

,Product,CustomerType,USD,EU
0,Car,Dealer,170,160
1,Car,Member,180,170
2,Car,Non-member,200,185
3,Motorcycle,Dealer,75,70
4,Motorcycle,Member,85,80
5,Motorcycle,Non-member,100,90


### Based on Product and CustomerType, display the USD amount for each combination

In [8]:
df.pivot(index='Product', columns='CustomerType', values='USD')

CustomerType,Dealer,Member,Non-member
Product,,,
Car,170,180,200
Motorcycle,75,85,100


### Based on CustomerType and Product, display the EU amount for each combination

In [9]:
df.pivot(index='CustomerType', columns='Product', values='EU')

Product,Car,Motorcycle
CustomerType,,
Dealer,160,70
Member,170,80
Non-member,185,90


### Based on CustomerType and Product, display both the USD and EU amount for each combination

In [10]:
df.pivot(index='CustomerType', columns='Product', values=['USD','EU'])

USD              EU           
Product       Car Motorcycle  Car Motorcycle
CustomerType                                
Dealer        170         75  160         70
Member        180         85  170         80
Non-member    200        100  185         90

or

In [11]:
df.pivot(index='CustomerType', columns='Product')

USD              EU           
Product       Car Motorcycle  Car Motorcycle
CustomerType                                
Dealer        170         75  160         70
Member        180         85  170         80
Non-member    200        100  185         90

### What happen if there are duplicate values for the specified columns(CustomerType)

In [12]:
table = OrderedDict((
    ("Product", ['Car', 'Car', 'Car', 'Motorcycle', 'Motorcycle', 'Motorcycle']),
    ('CustomerType',['Dealer', 'Member', 'Member', 'Dealer', 'Member', 'Non-member']),
    ('USD',  ['170', '180', '200', '75', '85', '100']),
    ('EU',   ['160', '170', '185', '70', '80', '90'])
))
df = pd.DataFrame(table)

In [13]:
df

,Product,CustomerType,USD,EU
0,Car,Dealer,170,160
1,Car,Member,180,170
2,Car,Member,200,185
3,Motorcycle,Dealer,75,70
4,Motorcycle,Member,85,80
5,Motorcycle,Non-member,100,90


In [14]:
df.pivot(index='Product', columns='CustomerType', values='USD')

ValueError: Index contains duplicate entries, cannot reshape

The pivot method can not know what should be the value of the corresponding value in the pivoted table. Thus it throws an exception with the following message:
**ValueError: Index contains duplicate entries, cannot reshape **

Hence, before calling pivot we need to ensure that our data does not have rows with duplicate values for the specified columns. If we can’t ensure this we may have to use the **pivot_table** method instead.

## Pivot_table()

The pivot_table method comes to solve this problem. It works like pivot, but it aggregates the values from rows with duplicate entries for the specified columns


In [15]:
df.head()

,Product,CustomerType,USD,EU
0,Car,Dealer,170,160
1,Car,Member,180,170
2,Car,Member,200,185
3,Motorcycle,Dealer,75,70
4,Motorcycle,Member,85,80


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
Product         6 non-null object
CustomerType    6 non-null object
USD             6 non-null object
EU              6 non-null object
dtypes: object(4)
memory usage: 272.0+ bytes


In [17]:
cols = ['USD','EU']
for col in cols:  # Iterate over chosen columns
    df[col] = pd.to_numeric(df[col])

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
Product         6 non-null object
CustomerType    6 non-null object
USD             6 non-null int64
EU              6 non-null int64
dtypes: int64(2), object(2)
memory usage: 272.0+ bytes


In [19]:
df.pivot_table(index='Product', columns='CustomerType', values='USD')

CustomerType,Dealer,Member,Non-member
Product,,,
Car,170.0,190.0,NaN
Motorcycle,75.0,85.0,100.0


In [20]:
df.pivot_table(index='Product', columns='CustomerType', values='USD', aggfunc=np.mean)

CustomerType,Dealer,Member,Non-member
Product,,,
Car,170.0,190.0,NaN
Motorcycle,75.0,85.0,100.0


In [21]:
df.pivot_table(index='Product', columns='CustomerType', values='USD', aggfunc=np.min)

CustomerType,Dealer,Member,Non-member
Product,,,
Car,170.0,180.0,NaN
Motorcycle,75.0,85.0,100.0


## Real life data set

In [22]:
df  = pd.read_csv('model_car_sales_data.csv',  encoding = "ISO-8859-1")
#df  = pd.read_csv('model_cars_sales_data.csv',  encoding = "latin")

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 25 columns):
ORDERNUMBER         2823 non-null int64
QUANTITYORDERED     2823 non-null int64
PRICEEACH           2823 non-null float64
ORDERLINENUMBER     2823 non-null int64
SALES               2823 non-null float64
ORDERDATE           2823 non-null object
STATUS              2823 non-null object
QTR_ID              2823 non-null int64
MONTH_ID            2823 non-null int64
YEAR_ID             2823 non-null int64
PRODUCTLINE         2823 non-null object
MSRP                2823 non-null int64
PRODUCTCODE         2823 non-null object
CUSTOMERNAME        2823 non-null object
PHONE               2823 non-null object
ADDRESSLINE1        2823 non-null object
ADDRESSLINE2        302 non-null object
CITY                2823 non-null object
STATE               1337 non-null object
POSTALCODE          2747 non-null object
COUNTRY             2823 non-null object
TERRITORY           1749 non-null obje

In [24]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [25]:
len(df)

2823

### Now let's reduce the complexity and size of this data set for learning purpose

In [26]:
df_sample = df.sample(400, random_state=1)

In [27]:
len(df_sample)

400

In [28]:
df= df_sample[['SALES','STATUS','QTR_ID','YEAR_ID','PRODUCTLINE','MSRP','COUNTRY','DEALSIZE']]

In [29]:
df.to_csv('sales_sample.csv')

In [30]:
df.head()

,SALES,STATUS,QTR_ID,YEAR_ID,PRODUCTLINE,MSRP,COUNTRY,DEALSIZE
1477,2299.0,Shipped,2,2004,Motorcycles,112,USA,Small
248,4029.0,Shipped,4,2003,Motorcycles,150,USA,Medium
1226,5600.5,Shipped,2,2004,Vintage Cars,105,USA,Medium
1573,1851.0,Shipped,1,2004,Motorcycles,76,Japan,Small
598,14082.8,On Hold,2,2005,Vintage Cars,170,USA,Large


## Let's transform some data

### For each PRODUCTLINE, what's the total sales revenue for each year ?

In [31]:
df.pivot_table(index=['PRODUCTLINE'], columns=['YEAR_ID'], \
               values=['SALES'], aggfunc=np.sum)

SALES                     
YEAR_ID                2003       2004      2005
PRODUCTLINE                                     
Classic Cars      234314.29  224290.87  84086.17
Motorcycles        63096.51   77576.49  18658.28
Planes             23473.91   61061.66  34202.19
Ships              36724.81   48654.95  27217.43
Trains             17472.71   19549.33   1193.04
Trucks and Buses   56799.21   53608.11  12944.15
Vintage Cars      101524.43  134375.88  64720.64

### For each PRODUCTLINE, what's the total sales revenue for each year and quarter ?

In [32]:
df.pivot_table(index=['PRODUCTLINE','YEAR_ID'], columns=['QTR_ID'], \
               values=['SALES'], aggfunc=np.sum)

SALES                               
QTR_ID                           1         2         3          4
PRODUCTLINE      YEAR_ID                                         
Classic Cars     2003     43501.01  37444.20  28747.06  124622.02
                 2004     51571.83  35255.45  68454.63   69008.96
                 2005     63750.82  20335.35       NaN        NaN
Motorcycles      2003      5951.72  11725.02   6128.82   39290.95
                 2004      1851.00  17319.28  14930.03   43476.18
                 2005     16455.80   2202.48       NaN        NaN
Planes           2003          NaN       NaN  11965.39   11508.52
                 2004     10307.99  20537.42   9939.49   20276.76
                 2005     27749.21   6452.98       NaN        NaN
Ships            2003          NaN   7691.65   6611.58   22421.58
                 2004     11561.67  14768.53   6559.06   15765.69
                 2005     19540.20   7677.23       NaN        NaN
Trains           2003      3227.63   3045.21   5283.76    5916.11
                 2004          NaN   5082.42       NaN   14466.91
                 2005          NaN   1193.04       NaN        NaN
Trucks and Buses 2003      4615.78   2925.09   7807.06   41451.28
                 2004      4716.23   8080.36  15902.57   24908.95
                 2005     12944.15       NaN       NaN        NaN
Vintage Cars     2003      8251.56  18348.78  10673.92   64250.17
                 2004     24934.51  43377.24  34584.04   31480.09
                 2005     45818.58  18902.06       NaN        NaN

### Explore parameter: **margins** 

boolean, default False.
Add all row / columns (e.g. for subtotal / grand totals)

In [33]:
df.pivot_table(index=['PRODUCTLINE','YEAR_ID'], columns=['QTR_ID'], \
               values=['SALES'], aggfunc=np.sum, margins=True)

SALES                                   \
QTR_ID                            1          2          3          4   
PRODUCTLINE      YEAR_ID                                               
Classic Cars     2003      43501.01   37444.20   28747.06  124622.02   
                 2004      51571.83   35255.45   68454.63   69008.96   
                 2005      63750.82   20335.35        NaN        NaN   
Motorcycles      2003       5951.72   11725.02    6128.82   39290.95   
                 2004       1851.00   17319.28   14930.03   43476.18   
                 2005      16455.80    2202.48        NaN        NaN   
Planes           2003           NaN        NaN   11965.39   11508.52   
                 2004      10307.99   20537.42    9939.49   20276.76   
                 2005      27749.21    6452.98        NaN        NaN   
Ships            2003           NaN    7691.65    6611.58   22421.58   
                 2004      11561.67   14768.53    6559.06   15765.69   
                 2005      19540.20    7677.23        NaN        NaN   
Trains           2003       3227.63    3045.21    5283.76    5916.11   
                 2004           NaN    5082.42        NaN   14466.91   
                 2005           NaN    1193.04        NaN        NaN   
Trucks and Buses 2003       4615.78    2925.09    7807.06   41451.28   
                 2004       4716.23    8080.36   15902.57   24908.95   
                 2005      12944.15        NaN        NaN        NaN   
Vintage Cars     2003       8251.56   18348.78   10673.92   64250.17   
                 2004      24934.51   43377.24   34584.04   31480.09   
                 2005      45818.58   18902.06        NaN        NaN   
All                       356749.69  282363.79  227587.41  528844.17   

                                      
QTR_ID                           All  
PRODUCTLINE      YEAR_ID              
Classic Cars     2003      234314.29  
                 2004      224290.87  
                 2005       84086.17  
Motorcycles      2003       63096.51  
                 2004       77576.49  
                 2005       18658.28  
Planes           2003       23473.91  
                 2004       61061.66  
                 2005       34202.19  
Ships            2003       36724.81  
                 2004       48654.95  
                 2005       27217.43  
Trains           2003       17472.71  
                 2004       19549.33  
                 2005        1193.04  
Trucks and Buses 2003       56799.21  
                 2004       53608.11  
                 2005       12944.15  
Vintage Cars     2003      101524.43  
                 2004      134375.88  
                 2005       64720.64  
All                       1395545.06

### Find the average sales dollar amount for each order based on DEALSIZE and PRODUCTLINE. 

In [34]:
df.pivot_table(index=['DEALSIZE','PRODUCTLINE'],values=['SALES'], aggfunc=np.mean)

SALES
DEALSIZE PRODUCTLINE                   
Large    Classic Cars       7893.528889
         Motorcycles        8691.705000
         Planes             7657.400000
         Vintage Cars      10008.270000
Medium   Classic Cars       4723.311500
         Motorcycles        4723.839444
         Planes             3829.020714
         Ships              3886.169474
         Trains             4299.806667
         Trucks and Buses   4577.473889
         Vintage Cars       4154.092917
Small    Classic Cars       2038.796739
         Motorcycles        2108.102222
         Planes             2075.694583
         Ships              2279.998235
         Trains             1773.748571
         Trucks and Buses   2275.385556
         Vintage Cars       1695.230476

## Problems

### For each PRODUCTLINE, what's the total sales revenue for each quarter ?

In [35]:
df.pivot_table(index=['PRODUCTLINE'], columns=['QTR_ID'], \
               values=['SALES'], aggfunc=np.sum)

SALES                               
QTR_ID                    1         2         3          4
PRODUCTLINE                                               
Classic Cars      158823.66  93035.00  97201.69  193630.98
Motorcycles        24258.52  31246.78  21058.85   82767.13
Planes             38057.20  26990.40  21904.88   31785.28
Ships              31101.87  30137.41  13170.64   38187.27
Trains              3227.63   9320.67   5283.76   20383.02
Trucks and Buses   22276.16  11005.45  23709.63   66360.23
Vintage Cars       79004.65  80628.08  45257.96   95730.26

In [36]:
df.pivot_table(index=['PRODUCTLINE'], columns=['QTR_ID'], \
               values=['SALES'], aggfunc=np.sum, margins=True)

SALES                                             
QTR_ID                    1          2          3          4         All
PRODUCTLINE                                                             
Classic Cars      158823.66   93035.00   97201.69  193630.98   542691.33
Motorcycles        24258.52   31246.78   21058.85   82767.13   159331.28
Planes             38057.20   26990.40   21904.88   31785.28   118737.76
Ships              31101.87   30137.41   13170.64   38187.27   112597.19
Trains              3227.63    9320.67    5283.76   20383.02    38215.08
Trucks and Buses   22276.16   11005.45   23709.63   66360.23   123351.47
Vintage Cars       79004.65   80628.08   45257.96   95730.26   300620.95
All               356749.69  282363.79  227587.41  528844.17  1395545.06

### For each PRODUCTLINE, find out the total sales dollar amount for each COUNTRY

In [37]:
df.pivot_table(index=['PRODUCTLINE','COUNTRY'], values=['SALES'], aggfunc=np.sum)

SALES
PRODUCTLINE      COUNTRY               
Classic Cars     Australia     37849.45
                 Austria        9333.38
                 Belgium        6144.60
                 Canada         9670.00
                 Denmark       16869.60
                 Finland       13824.02
                 France        58714.96
                 Germany       18320.05
                 Italy         22738.48
                 Norway        26372.18
                 Philippines   19090.63
                 Singapore     22110.17
                 Spain         71334.73
                 Sweden        11615.44
                 Switzerland   13104.52
                 UK            35388.24
                 USA          150210.88
Motorcycles      Australia     17245.32
                 Austria       16108.11
                 Finland        2194.17
                 France        15630.00
                 Japan          1851.00
                 Norway         7045.00
                 Spain         13095.52
                 Sweden         3988.60
                 USA           82173.56
Planes           Australia      4916.12
                 Canada         5227.25
                 Finland        8299.93
                 France        16556.31
...                                 ...
Ships            USA           50062.53
Trains           France         5924.47
                 Singapore      5396.57
                 Spain          9718.51
                 UK             2051.56
                 USA           15123.97
Trucks and Buses Australia     10766.22
                 Canada         7668.38
                 Denmark        3403.12
                 Finland        2701.92
                 France        23014.21
                 Spain         19916.73
                 Sweden         2086.92
                 UK             2174.42
                 USA           51619.55
Vintage Cars     Australia     34747.30
                 Austria        2687.04
                 Belgium       11814.24
                 Canada         4296.56
                 France        32181.46
                 Germany        4415.91
                 Italy         14797.35
                 Japan          1763.86
                 Norway        13452.80
                 Philippines    1935.09
                 Singapore      2835.93
                 Spain         48621.32
                 Sweden         1871.83
                 UK             6471.70
                 USA          118728.56

[72 rows x 1 columns]

### For each DEALSIZE and PRODUCTLINE, find out the total sales dollar amount for each STATUS

In [38]:
df.pivot_table(index=['DEALSIZE','PRODUCTLINE'],columns=['STATUS'],\
               values=['SALES'], aggfunc=np.sum)

SALES                                         \
STATUS                    Cancelled Disputed In Process   On Hold Resolved   
DEALSIZE PRODUCTLINE                                                         
Large    Classic Cars           NaN      NaN        NaN       NaN      NaN   
         Motorcycles            NaN      NaN        NaN       NaN      NaN   
         Planes                 NaN      NaN        NaN       NaN      NaN   
         Vintage Cars           NaN      NaN        NaN  14082.80      NaN   
Medium   Classic Cars      10383.78      NaN   10614.54   4421.34      NaN   
         Motorcycles            NaN      NaN        NaN       NaN      NaN   
         Planes                 NaN      NaN        NaN       NaN  3167.00   
         Ships              3162.95      NaN        NaN   3079.44      NaN   
         Trains             5082.42      NaN        NaN       NaN      NaN   
         Trucks and Buses       NaN      NaN        NaN       NaN      NaN   
         Vintage Cars      17830.95      NaN        NaN   4819.26  4196.07   
Small    Classic Cars       1706.80      NaN    2233.69       NaN      NaN   
         Motorcycles            NaN  2202.48        NaN       NaN      NaN   
         Planes             4147.47      NaN        NaN       NaN  2182.44   
         Ships                  NaN      NaN        NaN   2352.59      NaN   
         Trains                 NaN      NaN        NaN       NaN      NaN   
         Trucks and Buses       NaN      NaN        NaN   2086.92      NaN   
         Vintage Cars           NaN      NaN        NaN       NaN  1266.10   

                                      
STATUS                       Shipped  
DEALSIZE PRODUCTLINE                  
Large    Classic Cars       71041.76  
         Motorcycles        17383.41  
         Planes             15314.80  
         Vintage Cars       15942.01  
Medium   Classic Cars      352445.26  
         Motorcycles        85029.11  
         Planes             50439.29  
         Ships              67594.83  
         Trains             20716.42  
         Trucks and Buses   82394.53  
         Vintage Cars      172550.18  
Small    Classic Cars       89844.16  
         Motorcycles        54716.28  
         Planes             43486.76  
         Ships              36407.38  
         Trains             12416.24  
         Trucks and Buses   38870.02  
         Vintage Cars       69933.58

In [39]:
df.pivot_table(index=['DEALSIZE','PRODUCTLINE','STATUS'], \
               values=['SALES'], aggfunc=np.sum)

SALES
DEALSIZE PRODUCTLINE      STATUS               
Large    Classic Cars     Shipped      71041.76
         Motorcycles      Shipped      17383.41
         Planes           Shipped      15314.80
         Vintage Cars     On Hold      14082.80
                          Shipped      15942.01
Medium   Classic Cars     Cancelled    10383.78
                          In Process   10614.54
                          On Hold       4421.34
                          Shipped     352445.26
         Motorcycles      Shipped      85029.11
         Planes           Resolved      3167.00
                          Shipped      50439.29
         Ships            Cancelled     3162.95
                          On Hold       3079.44
                          Shipped      67594.83
         Trains           Cancelled     5082.42
                          Shipped      20716.42
         Trucks and Buses Shipped      82394.53
         Vintage Cars     Cancelled    17830.95
                          On Hold       4819.26
                          Resolved      4196.07
                          Shipped     172550.18
Small    Classic Cars     Cancelled     1706.80
                          In Process    2233.69
                          Shipped      89844.16
         Motorcycles      Disputed      2202.48
                          Shipped      54716.28
         Planes           Cancelled     4147.47
                          Resolved      2182.44
                          Shipped      43486.76
         Ships            On Hold       2352.59
                          Shipped      36407.38
         Trains           Shipped      12416.24
         Trucks and Buses On Hold       2086.92
                          Shipped      38870.02
         Vintage Cars     Resolved      1266.10
                          Shipped      69933.58

In [40]:
df_test = df.pivot_table(index=['DEALSIZE','PRODUCTLINE','STATUS'], \
               values=['SALES'], aggfunc=np.sum)

In [41]:
df_test.head(5)

SALES
DEALSIZE PRODUCTLINE  STATUS           
Large    Classic Cars Shipped  71041.76
         Motorcycles  Shipped  17383.41
         Planes       Shipped  15314.80
         Vintage Cars On Hold  14082.80
                      Shipped  15942.01

If need to remove MultiIndex, do a .reset_index()

In [42]:
df_test.reset_index().head()

,DEALSIZE,PRODUCTLINE,STATUS,SALES
0,Large,Classic Cars,Shipped,71041.76
1,Large,Motorcycles,Shipped,17383.41
2,Large,Planes,Shipped,15314.80
3,Large,Vintage Cars,On Hold,14082.80
4,Large,Vintage Cars,Shipped,15942.01


## Exercise: Use original model_car_sales_data.csv and explore what you can do to reshape the data.

In [ ]:
df  = pd.read_csv('model_car_sales_data.csv',  encoding = "ISO-8859-1")